### Q1: 
In  this  exercise  we  will  compute  the  PageRank  scores  for  pages  in  a  dataset  provided  byGoogle in a programming contest in 2002.  

The fileweb-Google_10k.txt contains data about 10,000 web pages and 78,323 links.  The file format is:
The first four rows contain metadata about the data set and are self-explained.
The following rows consist of two values that represent the link from the web page inthe first column to the web page in the second column.
For example, if the row is 0 11342, this means that there is a directed link from page id0 to page id 11342.

In [65]:
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True)

In [78]:
outlinks = {}  # key is the node and values are the page ids that it points to
inlinks = set() # set of all inlinks

with open('web-Google_10k.txt') as f:
    for line in f:
        if not line.startswith('#'):
            source, dest = line.split('\t')
            if int(source) not in outlinks:
                outlinks[int(source)] = {int(dest.strip())}
            else:
                outlinks[int(source)].add(int(dest.strip()))
                
                
            inlinks.add(int(dest))
            
      
print(len(inlinks))           
print(len(outlinks)) 

    

9896
8765


### 1.1 Find all the dead ends.  A page is a dead end if it has no out-going links.

In [79]:
dead_ends=np.array(list(inlinks - outlinks.keys()))

print("Total deadends are : ", len(dead_ends))
dead_ends

Total deadends are :  1235


array([696320, 196611, 786440, ...,  40927, 401378, 851963])

### 1.2 Build the page transition matrix A from the data set and make sure it is a stochastic matrix.

In [82]:
unique_nodes = list(set(list(inlinks) + list(outlinks)))
n = len(unique_nodes)
stoch_matrix = np.zeros((n, n))

# creating transition matrix
for i in range(len(unique_nodes)):
    for j in range(len(unique_nodes)):
        if unique_nodes[i] in outlinks:
            if unique_nodes[j] in outlinks[unique_nodes[i]]:
                stoch_matrix[i][j] = 1/len(outlinks[unique_nodes[i]])
                

# Handling dangling pages to form stochastic matrix

rows = np.where(~stoch_matrix.any(axis=1))[0]
print("Dangling pages : ", len(rows))
for i in rows:
    stoch_matrix[i][:] = 1/n
                

Dangling pages :  1235


In [83]:
stoch_matrix.shape

(10000, 10000)

### 1.3 Implement the PageRank algorithm.

In [84]:
def page_rank(A, d, n, eps):

    e = np.ones(n)
    p_new = e/n
    k = 0
    
    while(True):
        p_old = p_new
        p_new = d*np.matmul(A.T, p_old) + (1-d)*e/n
        
        if k%5==0:
            print("Iter : {} and Error : {} ".format(k, np.absolute(p_old-p_new).sum())) 
        
        k += 1
        
        if (np.absolute(p_old-p_new).sum() < eps):
            print("Iter : {} and Error : {} ".format(k, np.absolute(p_old-p_new).sum()))    
            return p_new
    

### 1.4 Run the PageRank algorithm on the matrix A with a damping factor of d= 0.85 and = 0.001.  The output format should be two-column:  the first column is the web pageid and the second column is its PageRank score.  Sort the output by descending order of the PageRank scores.

In [85]:
d = 0.85
eps = 0.001

p = page_rank(stoch_matrix, d, n, eps)


Iter : 0 and Error : 0.7674622280257468 
Iter : 5 and Error : 0.029884494881885312 
Iter : 10 and Error : 0.006911969665055052 
Iter : 15 and Error : 0.002208293271365266 
Iter : 20 and Error : 0.0009586197515640398 


In [86]:
p_df = pd.DataFrame(p, index=unique_nodes)
p_df = p_df.sort_values(by=[0], ascending=False)
p_df.head()

,0
486980,0.006931
285814,0.004733
226374,0.003388
163075,0.003339
555924,0.002695
